In [4]:
%%writefile EDA_warehouse_operational_efficiency.sql

# 01. generate the columns needed for visualization
CREATE OR REPLACE TABLE `portfolio-473815.warehouse_data.picking_eda_ready` AS
SELECT
  *,

  --  Picking Speed Category
  CASE
    WHEN picking_time_min < 10 THEN 'Fast'
    WHEN picking_time_min BETWEEN 10 AND 20 THEN 'Medium'
    ELSE 'Slow'
  END AS picking_speed_category,

  --  Efficiency Score (item per minute)
  ROUND(item_count / NULLIF(picking_time_min, 0), 2) AS order_efficiency_score,

  --  Quality Flag
  CASE
    WHEN quality_check_passed = TRUE THEN 'Passed'
    ELSE 'Failed'
  END AS quality_flag,

  --  Shipping Speed Group (berdasarkan logika bisnis umum)
  CASE
    WHEN shipping_method = 'Air' THEN 'Fast'
    WHEN shipping_method = 'Land' THEN 'Medium'
    WHEN shipping_method = 'Sea' THEN 'Slow'
    ELSE 'Unknown'
  END AS shipping_speed_group,

  --  Error status
  CASE
    WHEN error_type = 'No Error' THEN 'No Error'
    ELSE 'Error'
  END AS error_status,

  --  Shift Label
  CASE
    WHEN shift = 1 THEN 'Morning'
    WHEN shift = 2 THEN 'Evening'
    ELSE 'Unknown'
  END AS shift_label,

  --  Date Features
  EXTRACT(DAY FROM DATE(date)) AS day_of_month,
  FORMAT_DATE('%A', DATE(date)) AS weekday_name

FROM `portfolio-473815.warehouse_data.picking_clean`;

# 02. Korelation time and item
SELECT
  warehouse_zone,
  ROUND(CORR(item_count, picking_time_min), 3) AS corr_item_picking,
  ROUND(AVG(picking_time_min), 2) AS avg_picking_time,
  ROUND(AVG(item_count), 2) AS avg_item_count
FROM `portfolio-473815.warehouse_data.picking_clean`
GROUP BY warehouse_zone;

# 03. Error type distribution of USA Error Order
SELECT
  error_type,
  COUNT(*) AS total_error,
  ROUND(
    COUNT(error_type)* 100 / SUM(COUNT(error_type)) OVER(),
    2
  ) AS error_percentage_of_total
FROM `portfolio-473815.warehouse_data.picking_eda_ready`
WHERE error_type != 'No Error'
GROUP BY error_type
ORDER BY total_error DESC;

# 04. Error type distribution of warehouse D (USA and Error Order)
SELECT
  error_type,
  COUNT(*) AS total_error,
  ROUND(
    COUNT(error_type)* 100 / SUM(COUNT(error_type)) OVER(),
    2
  ) AS error_percentage_of_total
FROM `portfolio-473815.warehouse_data.picking_eda_ready`
WHERE error_type != 'No Error'and warehouse_zone = 'D'AND destination_country = 'Usa'
GROUP BY error_type
ORDER BY total_error DESC;

# 05. Error Percentage and efficiency score of Pickers
SELECT
  picker_id,
  ROUND(AVG(order_efficiency_score),2) as efficiency_score,
  ROUND(COUNTIF(error_status='Error')*100 / COUNT(order_id),2) as percentage_error
FROM `portfolio-473815.warehouse_data.picking_eda_ready`
GROUP BY picker_id
ORDER BY efficiency_score DESC;

# 06. Error Percentage and efficiency score of Pickers (USA, Error Order, Warehouse D)
SELECT
  picker_id,
  ROUND(AVG(order_efficiency_score),2) as efficiency_score,
  ROUND(COUNTIF(error_status='Error')*100 / COUNT(order_id),2) as percentage_error
FROM `portfolio-473815.warehouse_data.picking_eda_ready`
WHERE destination_country = 'Usa' and warehouse_zone = 'D'
GROUP BY picker_id
ORDER BY efficiency_score DESC;

# 07. Avg picking time and total order of picker (USA, Error order, warehouse D)
SELECT
  picker_id,
  ROUND(AVG(picking_time_min), 2) AS avg_picking_time,
  COUNT(order_id) AS total_error_order
FROM `portfolio-473815.warehouse_data.picking_eda_ready`
WHERE destination_country = 'Usa' AND error_status = 'Error' AND warehouse_zone = 'D'
GROUP BY picker_id
ORDER BY total_error_order DESC

# 08. Total order and error percentage of Weekday and shift (USA, Error Order, warehouse D)
SELECT
  weekday_name,
  shift_label,
  COUNT(order_id) AS total_orders,
  ROUND((COUNTIF(error_status = 'Error') / COUNT(order_id)) * 100, 2) AS error_percentage,
  ROUND(AVG(CASE WHEN error_status = 'Error' THEN order_efficiency_score END), 2) AS avg_eff_score_error
FROM `portfolio-473815.warehouse_data.picking_eda_ready`
WHERE destination_country = 'Usa'
  AND warehouse_zone = 'D'
GROUP BY weekday_name, shift_label
ORDER BY error_percentage DESC;

# 09. Error percentage of Shipping Method (USA, Error Order, warehouse D)
SELECT
  shipping_method,
  ROUND(COUNTIF(error_status = 'Error')*100 / COUNT(order_id),2) as error_percentage,
FROM `portfolio-473815.warehouse_data.picking_eda_ready`
WHERE destination_country = 'Usa' and warehouse_zone = 'D'
GROUP BY shipping_method
ORDER BY error_percentage DESC



Overwriting EDA_warehouse_operational_efficiency.sql
